First thing to do is get the texts from Project Gutenberg.

Use method from this website to get text IDs:

https://skeptric.com/gutenberg/

Keep this package in mind for tokenization:

https://pypi.org/project/semchunk/

In [2]:
import csv
from collections import Counter
from io import BytesIO, StringIO
from pathlib import Path
import requests
import pandas as pd
import gutenbergpy.textget
import json

In [15]:
# get catalog from Gutenberg
gut_cat = 'https://www.gutenberg.org/cache/epub/feeds/pg_catalog.csv.gz'

r = requests.get(gut_cat)
csv_text = r.content.decode('utf-8')

In [20]:
# get metadata for desired books
titles = ['Sense and Sensibility', 'Pride and Prejudice', 'Mansfield Park', 'Emma', 'Persuasion', 'Northanger Abbey', 'The Watsons: By Jane Austen, Concluded by L. Oulton']
author = 'Austen, Jane'

book_metadata = [book for book in csv.DictReader(StringIO(csv_text)) 
                 if author in book['Authors']
                 and book['Type'] == 'Text'
                 and book['Language'] == 'en'
                 and book['Title'] in titles]

In [21]:
# drop older versions of the text from metadata
old_versions = ['1342', '161']

book_metadata = [b for b in book_metadata if b['Text#'] not in old_versions]

In [24]:
# function to get book text using gutenbergpy
def book_text(id):
    r = gutenbergpy.textget.get_text_by_id(id)
    c = gutenbergpy.textget.strip_headers(r)
    return c

In [26]:
# save books as csv files in data folder
for book in book_metadata:
    id = book['Text#']
    print(id)
    text = book_text(id)
    filename = f'../data/{id}.txt'
    with open(filename, 'wt', encoding = 'utf-8') as book_file:
        book_file.write(str(text))

105
121
141
158
21839
42671
63569


In [27]:
# save metadata
with open('../data/austen_metadata.csv', 'wt') as f:
    csv_writer = csv.DictWriter(f, fieldnames = book_metadata[0].keys())
    csv_writer.writeheader()
    for b in book_metadata:
        csv_writer.writerow(b)